<a href="https://colab.research.google.com/github/318245915/Reconocimiento-de-Patrones-y-Aprendizaje-Automatizado/blob/main/Proyecto_Final_RDPYAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de recomendación de películas o música basado en preferencias y comportamientos pasados de los usuarios.

Obtención y preprocesamiento de datos.

1.   Importación de librerías: Importamos las librerías necesarias para manipulación de datos (pandas), división del dataset (train_test_split), y normalización de datos (StandardScaler).
2.   Cargar los datos: Leemos los archivos ratings.csv y movies.csv que contienen las calificaciones y la información de las películas, respectivamente.
3. Combinar los datos: Unimos los datasets de calificaciones y películas en base al movieId para tener un dataset completo con las calificaciones y los títulos de las películas.
4. División de datos: Dividimos el dataset combinado en conjuntos de entrenamiento y prueba.
5. Normalización de calificaciones: Normalizamos las calificaciones para mejorar el rendimiento del modelo.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
!pip install scikit-surprise

# Cargar los datos de MovieLens
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

# Verificar las primeras filas de los datasets
print(ratings.head())
print(movies.head())

# Unir las calificaciones con los títulos de las películas
data = pd.merge(ratings, movies, on='movieId')

# Mostrar algunas filas del dataset combinado
print(data.head())

# Preprocesar los datos
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Normalizar las calificaciones
scaler = StandardScaler()
train_data['rating'] = scaler.fit_transform(train_data[['rating']])
test_data['rating'] = scaler.transform(test_data[['rating']])

# Mostrar el tamaño de los conjuntos de entrenamiento y prueba
print(f'Tamaño del conjunto de entrenamiento: {train_data.shape}')
print(f'Tamaño del conjunto de prueba: {test_data.shape}')


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp             title  \
0       1        1     4.0   964982703  Toy Story (1995)   
1       5        1     4.0   847434962  Toy Story (1995) 

Diseño y entrenamiento del Modelo de Filtrado Colaborativo

1.   Configuración del lector: Utilizamos Reader de Surprise para especificar la escala de calificaciones.
2. Conversión de datos: Convertimos el DataFrame de entrenamiento en un formato que Surprise pueda manejar.
3. Definición del modelo: Utilizamos la técnica de descomposición en valores singulares (SVD) para el filtrado colaborativo.
4. Validación cruzada: Evaluamos el modelo utilizando validación cruzada para obtener métricas de rendimiento.
5. Entrenamiento final: Entrenamos el modelo en el conjunto completo de entrenamiento.


In [6]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Configurar el lector para Surprise
reader = Reader(rating_scale=(0.5, 5.0))

# Convertir el DataFrame en un formato adecuado para Surprise
data = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

# Definir el modelo de filtrado colaborativo utilizando SVD
model = SVD()

# Entrenar y evaluar el modelo utilizando validación cruzada
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Entrenar el modelo en el conjunto completo de entrenamiento
trainset = data.build_full_trainset()
model.fit(trainset)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1258  1.1139  1.1051  1.1086  1.1146  1.1136  0.0070  
MAE (testset)     0.8248  0.8213  0.8130  0.8081  0.8180  0.8171  0.0059  
Fit time          2.44    1.17    1.18    1.18    1.14    1.42    0.51    
Test time         0.10    0.19    0.09    0.09    0.08    0.11    0.04    


Evaluar el modelo

1.   Conversión del conjunto de prueba: Convertimos el conjunto de prueba en un formato que Surprise pueda manejar.
2. Predicciones: Utilizamos el modelo entrenado para realizar predicciones sobre el conjunto de prueba.
3. Evaluación: Calculamos el error cuadrático medio (RMSE) y el error absoluto medio (MAE) para evaluar el rendimiento del modelo.


In [7]:
from surprise import accuracy

# Convertir el conjunto de prueba en el formato adecuado para Surprise
test_data = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)
testset = test_data.build_full_trainset().build_testset()

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Evaluar el rendimiento del modelo
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


RMSE: 1.1060
MAE:  0.8111
RMSE: 1.1060396051868582
MAE: 0.8111296925774766


Desplegar el modelo


1.   Importación de Flask: Importamos Flask para crear la API.
2. Creación de la aplicación: Creamos una instancia de Flask.
3. Definición del endpoint: Definimos un endpoint /recommend que acepta el user_id como parámetro de consulta.
4. Obtención de recomendaciones: Para el usuario especificado, calculamos las predicciones de calificación para todas las películas no vistas y seleccionamos las 10 mejores recomendaciones.
5. Devolución de resultados: Convertimos las recomendaciones en formato JSON y las devolvemos al cliente.
6. Ejecución de la aplicación: Ejecutamos la aplicación Flask en modo de depuración.



In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    # Obtener las predicciones para todas las películas que el usuario no ha calificado
    all_movie_ids = data.build_full_trainset().all_items()
    predictions = [model.predict(user_id, movie_id) for movie_id in all_movie_ids]
    # Ordenar las predicciones por calificación estimada
    predictions.sort(key=lambda x: x.est, reverse=True)
    # Seleccionar las top 10 recomendaciones
    top_recommendations = predictions[:10]
    # Convertir las recomendaciones a un formato legible
    recommendations = [{'movieId': pred.iid, 'estimated_rating': pred.est} for pred in top_recommendations]
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
